In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
import pymysql
import json
import warnings
import DB


In [3]:
# db.cursor.execute('select * from team_record_merge')
# data = pd.DataFrame(db.cursor.fetchall())


In [4]:
# data = data.drop(3911) # 결측치 제거
# data = data.drop(3258)

In [5]:
def make_data(home, away):
  result = pd.DataFrame()
  # 어시스트 -> 어시스트 / 필드골 성공
  for idx in range(home.shape[0]):
    h_data, a_data = home.iloc[idx], away.iloc[idx]
    output_data = {}
    output_data['ast'] = h_data['ast'] / h_data['fgt']
    # 2점
    output_data['twop'] = h_data['fg'] / h_data['fgA']
    # 3점
    output_data['threep'] = h_data['threep'] / h_data['threepA']
    # 자유투
    output_data['freep'] = h_data['ft'] / h_data['ftA']
    # 공격 리바
    output_data['offr'] = h_data['offr'] / (h_data['offr'] + a_data['defr'])
    # 속공
    # output_data['tfb'] = h_data['tfb'] / h_data['fgt']
    # 벤치득점
    # output_data['bench'] = h_data['benchScoreCn'] / h_data['score']
    # 속공득점
    # output_data['tfbp'] = h_data['fbScoreCn'] / h_data['score']
    # 페인트존 득점
    output_data['ppp'] = h_data['pp'] / h_data['ppA']
    # 덩크
    output_data['dk'] = h_data['dk'] / h_data['dkA']
    # 최다 연속 득점 비율
    # output_data['maxContiScore'] = h_data['maxContiScoreCn'] / h_data['score']
    # 최다 리드 점수차
    # output_data['maxLeadScore'] = (h_data['score'] - a_data['score']) / h_data['maxLeadScoreCn']
    # 득점 우위 시간
    # output_data['scoreHighTime'] = h_data['scoreHighTimeCn'] / 2400
    # 세컨찬스 득점
    # output_data['secChanceScore'] = h_data['secChanceScoreCn'] / h_data['score']
    # 턴오버
    # output_data['to'] = (h_data['to'] + h_data['tto']) / (h_data['to'] + h_data['tto'] + a_data['to'] + a_data['tto'])
    # output_data['to'] = (h_data['to'] + h_data['tto']) / (h_data['to'] + h_data['tto'] + a_data['to'] + a_data['tto'])

    output_data['defr'] = h_data['defr'] / (a_data['offr'] + h_data['defr'])
    output_data['stl'] = h_data['stl'] / (h_data['stl'] + a_data['stl'])
    output_data['bs'] = h_data['bs'] / a_data['fgtA']
    output_data['foul'] = h_data['foul'] / (h_data['foul'] + a_data['foul'])
    output_data['tf'] = h_data['tf'] / (h_data['tf'] + a_data['tf'])
    output_data['gd'] = h_data['gd'] / (h_data['gd'] + a_data['gd'])

    output_data['score'] = h_data['score']
    # output_data['teamR'] = h_data['teamR'] / (h_data['teamR'] + a_data['teamR'])
    # result.append(output_data, ignore_index=True)
    # print(output_data)
    result = pd.concat([result, pd.DataFrame([output_data])], ignore_index=True)
  return result

In [6]:
def make_model(data):
    h_data = pd.DataFrame()
    a_data = pd.DataFrame()
    # 어시스트 -> 어시스트 / 필드골 성공
    h_data['ast'] = data['h_ast'] / data['h_fgt']
    a_data['ast'] = data['a_ast'] / data['a_fgt']
    # 2점
    h_data['twop'] = data['h_fg'] / data['h_fgA']
    a_data['twop'] = data['a_fg'] / data['a_fgA']

    # 3점
    h_data['threep'] = data['h_threep'] / data['h_threepA']
    a_data['threep'] = data['a_threep'] / data['a_threepA']

    # 자유투
    h_data['freep'] = data['h_ft'] / data['h_ftA']
    a_data['freep'] = data['a_ft'] / data['a_ftA']

    # 공격 리바
    h_data['offr'] = data['h_offr'] / (data['h_offr'] + data['a_defr'])
    a_data['offr'] = data['a_offr'] / (data['a_offr'] + data['h_defr'])
    # 속공
    h_data['tfb'] = data['h_tfb'] / data['h_fgt']
    a_data['tfb'] = data['a_tfb'] / data['a_fgt']

    # 벤치득점
    h_data['bench'] = data['h_benchScoreCn'] / data['h_score']
    a_data['bench'] = data['a_benchScoreCn'] / data['a_score']


    # 속공득점
    h_data['tfbp'] = data['h_fbScoreCn'] / data['h_score']
    a_data['tfbp'] = data['a_fbScoreCn'] / data['a_score']

    # 페인트존 득점
    h_data['ppp'] = data['h_pp'] / data['h_ppA']
    a_data['ppp'] = data['a_pp'] / data['a_ppA']

    # 덩크
    h_data['dk'] = data['h_dk'] / data['h_dkA']
    a_data['dk'] = data['a_dk'] / data['a_dkA']

    # 최다 연속 득점 비율
    h_data['maxContiScore'] = data['h_maxContiScoreCn'] / data['h_score']
    a_data['maxContiScore'] = data['a_maxContiScoreCn'] / data['a_score']

    # 최다 리드 점수차
    # h_data['maxLeadScore'] = (data['h_score'] - data['a_score']) / data['h_maxLeadScoreCn']
    # a_data['maxLeadScore'] = (data['a_score'] - data['h_score']) / data['a_maxLeadScoreCn']
    # h_data[h_data['maxLeadScore'] < -1]

    # 득점 우위 시간
    h_data['scoreHighTime'] = data['h_scoreHighTimeCn'] / 2400
    a_data['scoreHighTime'] = data['a_scoreHighTimeCn'] / 2400

    # 세컨찬스 득점
    h_data['secChanceScore'] = data['h_secChanceScoreCn'] / data['h_score']
    a_data['secChanceScore'] = data['a_secChanceScoreCn'] / data['a_score']

    # 턴오버
    h_data['to'] = (data['h_to'] + data['h_tto']) / (data['h_to'] + data['h_tto'] + data['a_to'] + data['a_tto'])
    a_data['to'] = (data['a_to'] + data['a_tto']) / (data['h_to'] + data['h_tto'] + data['a_to'] + data['a_tto'])

    h_data['defr'] = data['h_defr'] / (data['a_offr'] + data['h_defr'])
    a_data['defr'] = data['a_defr'] / (data['h_offr'] + data['a_defr'])
    h_data['stl'] = data['h_stl'] / (data['h_stl'] + data['a_stl'])
    a_data['stl'] = data['a_stl'] / (data['h_stl'] + data['a_stl'])
    h_data['bs'] = data['h_bs'] / data['a_fgtA']
    a_data['bs'] = data['a_bs'] / data['h_fgtA']
    h_data['foul'] = data['h_foul'] / (data['h_foul'] + data['a_foul'])
    a_data['foul'] = data['a_foul'] / (data['h_foul'] + data['a_foul'])
    h_data['tf'] = data['h_tf'] / (data['h_tf'] + data['a_tf'])
    a_data['tf'] = data['a_tf'] / (data['h_tf'] + data['a_tf'])
    h_data['gd'] = data['h_gd'] / (data['h_gd'] + data['a_gd'])
    a_data['gd'] = data['a_gd'] / (data['h_gd'] + data['a_gd'])
    h_data['teamR'] = data['h_teamR'] / (data['h_teamR'] + data['a_teamR'])
    a_data['teamR'] = data['a_teamR'] / (data['h_teamR'] + data['a_teamR'])

    h_data = h_data.fillna(0)
    h_data = h_data.replace([np.inf, -np.inf], 0)
    h_data['score'] = data['h_score']

    a_data = a_data.fillna(0)
    a_data = a_data.replace([np.inf, -np.inf], 0)
    a_data['score'] = data['a_score']

    return h_data.append(a_data, ignore_index=True)

In [7]:
#seasonKey : S19G01
def get_keyplayer(seasonKey, team):
    result = set()
    season = str(int(seasonKey[1:3]) -2)
    league = seasonKey[4:6]
    seasonKey = 'S' + season + 'G' + league
    db.cursor.execute('''SELECT *
                        FROM player_avg_record
                        WHERE seasonCode LIKE 'S{}%' and
                        pcode IN (SELECT pcode
                                    FROM player
                                    WHERE tcode = '{}' and seasonCode = '{}');'''.format(season, team, season))
    data = pd.DataFrame(db.cursor.fetchall())
    if data.size != 0:
        score_top_player = data.sort_values('score',ascending=False)['pcode'][:3]
        ast_top_player = data.sort_values('ast')['pcode'][:3]
        rb_top_player = data.sort_values('rb')['pcode'][:3]
        for player in score_top_player:
            result.add(player)
        # result.add(score_top_player[0])
        for player in ast_top_player:
            if player in result:
                continue
            result.add(player)
            # break
        for player in rb_top_player:
            if player in result:
                continue
            result.add(player)
            # break
        return result     
    else:
        return None

In [8]:
def get_season_keyplayer(season):
    # db = DB.MYDB()
    db.cursor.execute("select tcode from team_code")
    tcodes = db.cursor.fetchall()
    result = dict()
    for tcode in tcodes:
        tcode = tcode['tcode']
        if tcode == '6':
            tcode = '06'
        db.cursor.execute('''SELECT *
                        FROM player_avg_record
                        WHERE seasonCode LIKE 'S{}%' and
                        pcode IN (SELECT pcode
                                    FROM player
                                    WHERE tcode = '{}' and seasonCode = '{}');'''.format(season - 2, tcode, season))
        data = pd.DataFrame(db.cursor.fetchall())
        if data.size != 0:
            score_top_player = data.sort_values('score',ascending=False)['pcode'][:3]
            ast_top_player = data.sort_values('ast', ascending=False)['pcode'][:3]
            rb_top_player = data.sort_values('rb', ascending=False)['pcode'][:3]
            result[tcode] = set(score_top_player) | set(ast_top_player) | set(rb_top_player)
    return result

In [9]:
def get_recent(players, gameCode):
    df = pd.DataFrame()
    for player in players:
        db.cursor.execute("""
                        SELECT b.*, a.gameDate 
                        FROM game_meta a, player_record b
                        WHERE a.gmkey = b.gmkey and b.pcode = '{}' and b.gmkey < '{}'
                        ORDER BY gameDate desc
                        LIMIT 5""".format(player, gameCode))
        temp = pd.DataFrame(db.cursor.fetchall())
        df = pd.concat([df, temp])
    return df
        

In [13]:
db = DB.MYDB()
db.cursor.execute("SELECT gmkey, tcodeA, tcodeH FROM hygp.game_meta where gmkey > 'S19G01N1';")
records = db.cursor.fetchall()
db.cursor.execute("SELECT gmkey, h_score, a_score FROM hygp.team_record_merge;")
scores = pd.DataFrame(db.cursor.fetchall())
season_keyplayer = dict()
for season in range(17, 41):
    season_keyplayer[season] = get_season_keyplayer(season)
    

Connect!


In [11]:
def player_model():
    h_data = []
    a_data = []
    for idx, record in enumerate(records):
        gmkey = record['gmkey']
        season = int(gmkey[1:3])
        score = scores[scores['gmkey'] == gmkey]
        temp1 = dict(get_recent(season_keyplayer[season][record['tcodeH']], gmkey).mean(numeric_only=True))
        temp1['score'] = score['h_score']
        temp2 = dict(get_recent(season_keyplayer[season][record['tcodeA']], gmkey).mean(numeric_only=True))
        temp2['score'] = score['a_score']
        h_data.append(temp1)
        a_data.append(temp2)
        if idx % 100 == 0:
            print(gmkey)
            
        # h_df = pd.DataFrame(get_recent(home_keyplayer, gmkey).mean(numeric_only=True)).transpose()
        # a_df = pd.DataFrame(get_recent(away_keyplayer, gmkey).mean(numeric_only=True)).transpose()
        # if h_df.size == 0 or a_df.size == 0:
            # print("recent error :", gmkey)
        # h_df['score'] = score['h_score']
        # if a_df is None:
            # print(gmkey, away_keyplayer)
        # a_df['score'] = score['a_score']
        # model = pd.concat([model, make_data(h_df, a_df), make_data(a_df, h_df)])
        # print(gmkey)
    h_df = pd.DataFrame(h_data).to_csv("h_data.csv")
    a_df = pd.DataFrame(h_data).to_csv("a_data.csv")
    model = pd.concat([make_data(h_data, a_data), make_data(a_data, h_data)])
    return model


In [12]:
warnings.filterwarnings(action='ignore')
data = player_model()
data

S23G01N2
S23G01N2
S23G01N3
S23G01N4
S23G01N5
S23G01N6
S23G01N7
S23G01N8
S23G01N9
S23G01N10


KeyboardInterrupt: 

In [ ]:
data.to_csv("player_model")

In [ ]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
y_predict = model.predict(X_test)
mean_squared_error(y_predict, y_test, squared = False)
model.score(X_test, y_test)